---
## Setup
*   If you have not run the `FRCNN.ipynb` and the `SORT.ipynb` notebooks, those are the prior steps in the preprocessing pipeline, please run them before.
*   This setup is just imports and result extraction from the previous stage.
---

### CoLab setup

In [ ]:
ROOT = '/content/drive'
from google.colab import drive
drive.mount(ROOT)

Mounted at /content/drive


### Imports

In [ ]:
import os
import torch
import shutil
import subprocess
from tqdm import tqdm
from os.path import basename, splitext


HOME_PATH='./'
OPENPOSE_PATH="./openpose/"
FINAL_PATH = "/content/final/"
KEYPOINTS_PATH = "/content/keypoints/"
RAW_VIDEOS_PATH = "/content/drive/MyDrive/ataxia_dataset/"
# use CUDA if available
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

#### Extract output from previous phase

In [ ]:
# extract final frames
shutil.copy(RAW_VIDEOS_PATH + "final_frames.zip", "/content/")
!unzip final_frames.zip
!mv content/final/ /content/

---
## 5. KeyPoint extraction using OpenPose
*   Now we will extract keypoints from all the frames using OpenPose, and store these as well.
*   Run on a GPU runtime, takes about 3 hours.
---

### Installing OpenPose

In [ ]:
# see: https://github.com/CMU-Perceptual-Computing-Lab/openpose/issues/949
# install new CMake becaue of CUDA10
!wget -q https://cmake.org/files/v3.17/cmake-3.17.2-Linux-x86_64.tar.gz
!tar xfz cmake-3.17.2-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local

# install system dependencies
!apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev

git_repo_url = 'https://github.com/CMU-Perceptual-Computing-Lab/openpose.git'
project_name = splitext(basename(git_repo_url))[0]
!rm -rf openpose
# clone openpose
!git clone -q --depth 1 $git_repo_url
# --recursive necessary in the line below, as otherwise you can (sometimes) get "lpthreads" errors in cmake ("undefined reference to `pthread_create'" etc). See, for example, https://github.com/facebookarchive/caffe2/issues/1234
!sed -i 's/execute_process(COMMAND git checkout --recursive master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt
!cd openpose && git submodule update --init --recursive --remote

!wget "https://drive.usercontent.google.com/download?id=1QCSxJZpnWvM00hx49CJ2zky7PWGzpcEh&export=download&authuser=0&confirm=t&uuid=50355e54-0445-4875-9d0e-3866a1f6239d&at=APZUnTVA3sHsK5rk-u2O6phIms2L%3A1711987634035" -O models.zip

!unzip models.zip -d openpose

In [ ]:
!sed -i 's/executeShInItsFolder "getModels.sh"/# executeShInItsFolder "getModels.sh"/g' ./openpose/scripts/ubuntu/install_openpose_JetsonTX2_JetPack3.1.sh
!sed -i 's/executeShInItsFolder "getModels.sh"/# executeShInItsFolder "getModels.sh"/g' ./openpose/scripts/ubuntu/install_openpose_JetsonTX2_JetPack3.3.sh
!sed -i 's/download_model("BODY_25"/# download_model("BODY_25"/g' ./openpose/CMakeLists.txt
!sed -i 's/78287B57CF85FA89C03F1393D368E5B7/# 78287B57CF85FA89C03F1393D368E5B7/g' ./openpose/CMakeLists.txt
!sed -i 's/download_model("body (COCO)"/# download_model("body (COCO)"/g' ./openpose/CMakeLists.txt
!sed -i 's/5156d31f670511fce9b4e28b403f2939/# 5156d31f670511fce9b4e28b403f2939/g' ./openpose/CMakeLists.txt
!sed -i 's/download_model("body (MPI)"/# download_model("body (MPI)"/g' ./openpose/CMakeLists.txt
!sed -i 's/2ca0990c7562bd7ae03f3f54afa96e00/# 2ca0990c7562bd7ae03f3f54afa96e00/g' ./openpose/CMakeLists.txt
!sed -i 's/download_model("face"/# download_model("face"/g' ./openpose/CMakeLists.txt
!sed -i 's/e747180d728fa4e4418c465828384333/# e747180d728fa4e4418c465828384333/g' ./openpose/CMakeLists.txt
!sed -i 's/download_model("hand"/# download_model("hand"/g' ./openpose/CMakeLists.txt
!sed -i 's/a82cfc3fea7c62f159e11bd3674c1531/# a82cfc3fea7c62f159e11bd3674c1531/g' ./openpose/CMakeLists.txt

### Make (takes \~ 10 minutes!)

In [ ]:
!cd openpose && rm -rf build || true && mkdir build

In [ ]:
cmake_file='/content/openpose/CMakeLists.txt'
!cd openpose && sed -i 's/-DBUILD_python=OFF/-DBUILD_python=ON/g' $cmake_file
!cd openpose && sed -i 's/-DBUILD_python_layer=OFF/-DBUILD_python_layer=ON/g' $cmake_file
!cd openpose && sed -i 's/option(BUILD_PYTHON "Build OpenPose python." OFF)/option(BUILD_PYTHON "OpenPose python." ON)\noption(BUILD_BIN_FOLDER "Copy 3rd-party DLL files." ON)/g' $cmake_file

In [ ]:
!cd openpose && cd build && cmake .. -DUSE_CUDNN=OFF -DGENERATE_PYTHON_BINDINGS:BOOL="1" -DPYTHON_LIBRARY='/usr/lib/x86_64-linux-gnu/libpython3.10.so' && make -j`nproc`

### Make the build executable

In [ ]:
!cd $OPENPOSE_PATH && chmod -R 755 './build/'

### Extract keypoints

In [ ]:
!mkdir "$KEYPOINTS_PATH"

In [ ]:
# Using these frames,
for video in tqdm(os.listdir(FINAL_PATH)):
  # Extract keypoints using openpose
  if os.path.exists(f"/content/{KEYPOINTS_PATH}/{video[:-4]}/"):
    continue
  os.mkdir(f"/content/{KEYPOINTS_PATH}/{video[:-4]}/")
  track_label = os.listdir(f"/content/{FINAL_PATH}/{video[:-4]}/")[0]
  openposed = ["./build/examples/openpose/openpose.bin",
                "--image_dir", f"/content/{FINAL_PATH}/{video[:-4]}/{track_label}/", \
                "--number_people_max", "1", \
                "--write_json", f"/content/{KEYPOINTS_PATH}/{video[:-4]}/", \
                "--keypoint_scale", "3", \
                "--display", "0", \
                "--render_pose", "0", \
                "--model_pose", "BODY_25",]
  # need to run this from the openpose dir
  openpoze = subprocess.run(openposed, stderr=subprocess.PIPE, stdout=subprocess.PIPE, cwd="/content/openpose")
  print(openpoze.stderr)

In [ ]:
!zip keypoints.zip -r "$KEYPOINTS_PATH"
shutil.copy("keypoints.zip", RAW_VIDEOS_PATH) # the keypoints will be in your drive.

## Thank you!
This is all on the extraction side! We now have keypoints of each patient, we will now form gait cycles from these keypoints and train out network.
*   We had manually inspected the videos by overlaying these keypoints and unfortunately, some of them (~5-6) do have problems,
*   For such cases, we manually cropped the patient and ran openpose on those videos,
*   This happens because the Auto-Gait paper's release has blurred many details and these videos are out-of-distribution for several networks.